In [1]:
import ROOT
import os
import numpy as np
from lorenzetti_utils import dataframe_h
ROOT.gROOT.ProcessLine(dataframe_h)
ROOT.EnableImplicitMT()

Module libc not found.


Welcome to JupyROOT 6.28/02


In [2]:
filepath = '/home/lucas.nunes/data/lorenzetti/Zee.AOD.root'
tfile = ROOT.TFile(filepath)
tree = tfile.Get('physics')
# chain.Add(filepath)
tree.GetEntries()

9998

In [3]:
def get_deltaR(cluster_a, cluster_b):
    deta = cluster_a.eta - cluster_b.eta
    dphi = cluster_a.phi - cluster_b.phi
    delta_r = np.sqrt((deta**2) + (dphi**2))
    return delta_r

In [4]:
@ROOT.Numba.Declare(['xAOD::CaloCluster_t'], "float")
def get_cluster_deltaR(clusters_container):
    sublead = None
    lead = None
    sublead_et = -np.inf
    lead_et = -np.inf
    for cluster in clusters_container:
        if cluster.et >= lead_et:
            sublead = lead
            lead = cluster
            sublead_et = lead_et
            lead_et = cluster.et
        elif cluster.et >= sublead_et:
            sublead = cluster
            sublead_et = cluster.et

    return get_deltaR(lead, sublead)

Exception: Type xAOD::CaloCluster_t is not supported for jitting with numba. Valid fundamental types and RVecs thereof are ['float', 'double', 'int', 'unsigned int', 'long', 'unsigned long', 'bool']

In [ ]:
for i, event in enumerate(tree):
    event.deltaR = get_cluster_deltaR(event.CaloClusterContainer_Clusters)
    break

In [ ]:
@ROOT.Numba.Declare(['RVec<float>'], "float")
def biggest_energy(energy):
    return max(energy)

In [ ]:
rdf = ROOT.RDataFrame(tree)
new_rdf = rdf.Define('biggest_energy', 'Numba::biggest_energy(CaloClusterContainer_Clusters)')
new_rdf.Describe()

In [ ]:
new_rdf.Snapshot('tree', 'snapshot_test.root', 'biggest_energy')